# Ansible Roles
---
Roles in Ansible are tasks that are grouped together to allow for code (i.e. task) reuse. These roles are repositories of code that serve specific and often isolated purposes. For example, you may write a role to provision a VM or a role to perform a backup of VM disks. If you'd like, you can think of roles in Ansible like functions in other languages: something you can call over and over again that will perform a specific set of tasks and you alter the output or outcome just by the adjusting the input variables.


### Role Structure
An Ansible Role is a repo that, while not held to a strict construction standard, will often follow a similar structure regardless of the purpose it serves. At Chevron, an Ansible Role will almost always have the following structure:

![Role Structure](./notebook-files/role.png)

## Demos
---

### Using Roles

Similar to the modules, roles are a way to collect and unify functionality to be automated. They can be called in multiple ways, the two most common can be seen in the below example:

> Build the tasks/mail.yml below so you can see  the contents of the tasks that the role executes.
> Execute the playbook that calls the role in two ways
> - First as a role under the roles section
> - Second as a include_role under the tasks exction


In [ ]:
cat << EOF > "/home/cvx_admin_user/notebooks/playbooks/roles/debug-role/tasks/main.yml"
---
# tasks file for debug-role
- name: Output the value of the debug-role variable
  debug:
    var: string_variable
...
EOF

In [ ]:
wpb "role-playbook.yml"

########################  PLAYBOOK  ########################
---
- hosts: localhost
  connection: local

  roles:
    - role: debug-role
      string_variable: "Hello from the first call of the Debug Role"

  tasks:
    - include_role:
        name: debug-role
      vars:
        string_variable: "Hello from the second call of the Debug Role"
...
############################################################

ansible-playbook "role-playbook.yml"

### Extra Variables

We have seen a simple way of executing the playbook with the variables inside the playbook, but users can also override internal variables with their own variables through the command line and extra vars. What we are doing on the command line below is the same thing that ADO does with `extra_ansible_vars` in the build files.

> Run the playbook below 3 times and see how extra vars take precedence over the variables in the playbook
> - Build the extra vars file.
> - Run it with the variable set in the playbook
> - Uncomment and run it with extra variable value set on the command line -e "string_variable='Create a Linux Machine'"
> - Uncomment and run it withe the extra vars file using -e @/home/cvx_admin_user/extra_vars.yml


In [ ]:
cat << EOF > "/home/cvx_admin_user/extra_vars.yml"

string_variable: "Create a Very Large Linux Machine"

EOF


In [ ]:
wpb "role-playbook.yml"

########################  PLAYBOOK  ########################
---
- hosts: localhost
  connection: local
  gather_facts: false

  roles:
    - role: debug-role
      string_variable: "Create a Windows Machine"
...
############################################################

ansible-playbook "/home/cvx_admin_user/notebooks/playbooks/role-playbook.yml"
#ansible-playbook "/home/cvx_admin_user/notebooks/playbooks/role-playbook.yml" -e "string_variable='Create a Linux Machine'"
# ansible-playbook "/home/cvx_admin_user/notebooks/playbooks/role-playbook.yml" -e @/home/cvx_admin_user/extra_vars.yml

### Example playbook... again!

Below you can see a playbook that is quite similar to one we've already seen. That's becasue it's the exact same! Here we will see that a simple one file playbook can easily be turned into a role. 

#### tasks/main.yml in the role
```yaml
---
  - name: install sosreport package
    package:
      name: sosreport
      state: latest
    become: true
    
  - name: run sosreport
    command: "sos report -l"
    register: sosreport_output
    become: true
    no_log: true
    
  - name: print the output
    debug:
      msg: "{{ sosreport_output.stdout_lines }}"
    when: sosreport_debug
    
  - name: email me the results
    mail:
      host: localhost
      port: 25
      to: "{{ my_email_address }}"
      subject: "SOS Report for {{ ansible_default_ipv4.address }} on {{ ansible_date_time.date }}"
      body: "{{ sosreport_output.stdout_lines | to_nice_json }}"
```

> Note:
> - The output is the same as when we ran it previously
> - Rebuild the tasks/mail.yml so you can see the contents and what goes into it.
> - You can change the variables that are being passed into the role

In [ ]:
cat << EOF > "/home/cvx_admin_user/notebooks/playbooks/roles/sos-report-role/tasks/main.yml"
---
- name: install sosreport package
  package:
    name: sosreport
    state: latest
  become: true

- name: run sosreport
  command: "sos report -l"
  register: sosreport_output
  become: true
  no_log: true

- name: print the output
  debug:
    msg: "{{ sosreport_output.stdout_lines }}"
  when: sosreport_debug

- name: email me the results
  mail:
    host: localhost
    port: 25
    to: "{{ my_email_address }}"
    subject: "SOS Report for {{ ansible_default_ipv4.address }} on {{ ansible_date_time.date }}"
    body: "{{ sosreport_output.stdout_lines | to_nice_json }}"

EOF

In [ ]:
wpb "role-playbook.yml"

########################  PLAYBOOK  ########################
---
- hosts: localhost
  connection: local

  roles:
    - role: sos-report-role
      sosreport_debug: True
      my_email_address: "nobody@localhost"
...
############################################################

ansible-playbook "role-playbook.yml"

## Further Explanation
---

**Order of Operations:**
1. All dependencies listed in `meta/main.yml` will get run
2. All tasks in `tasks/main.yml` will get run
3. The variable order of precedence matters.  Extra vars overrides all.

**See Also:**
- [Roles Basics](https://docs.ansible.com/ansible/latest/user_guide/playbooks_reuse_roles.html)
- [Using Roles](https://docs.ansible.com/ansible/latest/user_guide/playbooks_reuse_roles.html#using-roles)